# Tutorial3: estimate 3D emission from image-plane directly

---
This tutorial demonstrates the recovery of 3D emission from a gravitationally lensed image plane. The unknown emission is modeled using a coordinate-based neural network (NeRF).

In [1]:
import bhnerf
import numpy as np
import os
from datetime import datetime
from astropy import units

# Runing on 2 GPUs
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

/srv/tmp/kyle/miniconda3/envs/jax/lib/python3.9/site-packages/ehtim/__init__.py:56: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


Welcome to eht-imaging! v 1.2.4 



In [2]:
"""
Generate gravitationally lensed image-plane of a hot-spot
"""
fov_M = 16.0
spin = 0.2
inclination = np.deg2rad(60.0)      
nt = 64                            

t_frames = np.linspace(0, 1.0, nt) * units.hr

# Compute geodesics (see Tutorial1)
geos = bhnerf.kgeo.image_plane_geos(
    spin, inclination, 
    num_alpha=64, num_beta=64, 
    alpha_range=[-fov_M/2, fov_M/2],
    beta_range=[-fov_M/2, fov_M/2]
)
t_injection = -float(geos.r_o)

# Keplerian prograde velocity field
Omega = np.sign(spin + np.finfo(float).eps) * np.sqrt(geos.M) / (geos.r**(3/2) + geos.spin * np.sqrt(geos.M))

emission_0 = bhnerf.emission.generate_hotspot_xr(
    resolution=(64, 64, 64), 
    rot_axis=[0.0, 0.0, 1.0], 
    rot_angle=0.0,
    orbit_radius=5.5,
    std=0.7,
    r_isco=bhnerf.constants.isco_pro(spin),
    fov=(fov_M, 'GM/c^2')
)
image_plane = bhnerf.emission.image_plane_dynamics(emission_0, geos, Omega, t_frames, t_injection)

/srv/tmp/kyle/miniconda3/envs/jax/lib/python3.9/site-packages/xarray/core/computation.py:821: RuntimeWarning: invalid value encountered in sqrt
  result_data = func(*input_data)


In [7]:
"""
Visualize the recovered 3D emission
This visualization requires ipyvolume: https://ipyvolume.readthedocs.io/en/latest/
"""
emission_estimate = bhnerf.network.sample_3d_grid(predictor.apply, optimizer.params, fov=fov_M)
bhnerf.visualization.ipyvolume_3d(emission_estimate, fov=fov_M, level=[0, 0.2, 0.7])

Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…